## Предсказание дохода от показа фильмов по базе TMDB (соревнование kaggle https://www.kaggle.com/c/tmdb-box-office-prediction/overview/evaluation) 

В рамках итоговой работы по курсу попробуем предсказать выручку от показа фильмов по данным базы TMDB.



## Постановка задачи

* Целевая переменная - кассовый доход для каждого фильма (revenue)
* Задача регрессии
* Метрика для оценки качества - среднеквадратичная логарифмическая ошибка (RMSLE)

Задача - предсказать международный кассовый доход для каждого фильма. Для каждого id из тестовых наборов необходимо предсказать значение переменной дохода.

Заявки оцениваются на основе среднеквадратичной логарифмической ошибки (RMSLE) между прогнозируемым значением и фактическим доходом.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
import os
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

## Загрузка данных

In [ ]:
train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')
dataset = pd.concat([train, test], ignore_index=True, sort=True)

## Анализ данных

In [ ]:
#Посмотрим на исходные данные
print(train.shape)
print(test.shape)
train.head(2)

In [ ]:
print(dataset.shape)
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.head(2)

In [ ]:
test.isnull().sum()

In [ ]:
train.nunique()

In [ ]:
train['release_month'] = train.release_date.str.extract('(\S+)/\S+/\S+', expand=False).astype(np.int16)
train['release_year'] = train.release_date.str.extract('\S+/\S+/(\S+)', expand=False).astype(np.int16)
train['release_day'] = train.release_date.str.extract('\S+/(\S+)/\S+', expand=False).astype(np.int16)
train.loc[(21 <= train.release_year) & (train.release_year <= 99), 'release_year'] += 1900
train.loc[train.release_year < 21, 'release_year'] += 2000

train['release_date'] = pd.to_datetime(train.release_day.astype(str) + '-' + 
                                       train.release_month.astype(str) + '-' + 
                                       train.release_year.astype(str))

train['release_weekday'] = train.release_date.dt.weekday_name.str.slice(0, 3)

In [ ]:
print(train.loc[15])

In [ ]:
#Посмотрим на зависимость некоторых признаков и целевой переменной, как она распределена
features = ['budget', 'popularity', 'runtime', 'revenue']
sns.pairplot(train[features])
plt.show()

In [ ]:
sns.distplot(train['revenue']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['revenue'].skew())
print("Kurtosis: %f" % train['revenue'].kurt())

Видим у целевой переменной длинный хвост и ассиметрию. Поэтому мы делаем логарифмическое преобразование

In [ ]:
train['revenue_log'] = np.log(train['revenue']+1)
sns.distplot(train['revenue_log']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['revenue_log'].skew())
print("Kurtosis: %f" % train['revenue_log'].kurt())

In [ ]:
sns.distplot(train['budget']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['revenue'].skew())
print("Kurtosis: %f" % train['revenue'].kurt())

В бюджете фильмов много нулевых значений заменим их на среднее значение

In [ ]:
#mean_budget = train['budget'].mean()
#print(mean_budget)
#train['budget'] = train['budget'].apply(lambda x: mean_budget if x == 0 else x)

Приведем к нормальному распределению

In [ ]:
train['budget_log'] = np.log(train['budget']+1)
sns.distplot(train['budget_log']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['budget_log'].skew())
print("Kurtosis: %f" % train['budget_log'].kurt())

In [ ]:
sns.distplot(train['popularity']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['revenue'].skew())
print("Kurtosis: %f" % train['revenue'].kurt())

In [ ]:
train['popularity_log'] = np.log(train['popularity'])
sns.distplot(train['popularity_log']);
#Асимметрия и Эксцесс
print("Skewness: %f" % train['popularity_log'].skew())
print("Kurtosis: %f" % train['popularity_log'].kurt())

In [ ]:
sns.regplot(train['budget'], train['revenue'])

In [ ]:
sns.regplot(train['budget_log'], train['revenue_log'])

In [ ]:
sns.regplot(train['popularity_log'], train['revenue_log'])

In [ ]:
fig = sns.countplot(train.release_weekday, order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
fig.set(ylabel='number of movies')
plt.title("Количество релизов фильмов в зависимости от дня недели")
plt.show()

In [ ]:
revenue_by_weekday = train.groupby('release_weekday')['revenue'].aggregate([np.sum])
revenue_by_weekday.reset_index(inplace=True)
fig =sns.barplot(x='release_weekday', y='sum', data=revenue_by_weekday, order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
fig.set(ylabel='Сумма выручки')
plt.title("Зависимость выручки от дня недели")
plt.show()

In [ ]:
fig = sns.countplot(train.release_month)
fig.set(ylabel='number of movies')
plt.show()

In [ ]:
revenue_by_month = train.groupby('release_month')['revenue'].aggregate([np.sum])
revenue_by_month.reset_index(inplace=True)
fig =sns.barplot(x='release_month', y='sum', data=revenue_by_month)
fig.set(ylabel='Сумма выручки')
plt.show()

**Делаем преобразования на общем датасете**

In [ ]:
#mean_budget = dataset['budget'].mean()
#dataset['budget'] = dataset['budget'].apply(lambda x: mean_budget if x == 0 else x)

In [ ]:
dataset['budget_log'] = np.log(dataset['budget']+1)
dataset['revenue_log'] = np.log(dataset['revenue'])
dataset['popularity_log'] = np.log(dataset['popularity'])
dataset = dataset.drop(["budget", "revenue", "popularity"], axis=1)

In [ ]:
dataset['budget_log']

In [ ]:
print(dataset.info())
print(dataset.isnull().sum())

In [ ]:
#заменим в тесте пустую дату
dataset.loc[dataset.release_date.isnull(), 'release_date'] = '01/01/2000'

In [ ]:
mean_runtime = dataset['runtime'].mean()
dataset.loc[dataset.runtime.isnull(), 'runtime'] = mean_runtime

In [ ]:
#удаляем не значащие поля
remove_list = ["id", "belongs_to_collection", "imdb_id", "poster_path", "crew", "overview", "status", "original_title", "tagline", "title", "homepage", "Keywords", "cast"]
dataset = dataset.drop(remove_list, axis=1)

In [ ]:
print(dataset.loc[15])
print(dataset.isnull().sum())

In [ ]:
#Вытащим из JSON значения
# 'genres', 'production_companies', 'production_countries' and 'spoken_languages'
threshold = 80
#dataset = train
for feature in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    dataset.loc[dataset[feature].isnull(), feature] = '{}'
    dataset[feature] = dataset[feature].apply(lambda x: sorted([d['name'] for d in eval(x)]))
    dataset['num_of_' + feature] = dataset[feature].apply(lambda x: len(x))
    dataset[feature] = dataset[feature].apply(lambda x: ','.join(map(str, x)))
    
    tmp = dataset[feature].str.get_dummies(sep=',')
    
    tmp = tmp.loc[:, tmp.sum() > threshold]
    dataset = pd.concat([dataset, tmp], axis=1)
    

In [ ]:
print(dataset.iloc[16])
print(dataset.shape)

In [ ]:
remove_list2 = ["genres", "original_language", "production_companies", "production_countries", "release_date", "spoken_languages"]
dataset = dataset.drop(remove_list2, axis=1)

In [ ]:
print(dataset.loc[16])
print(dataset.shape)

In [ ]:
train_num = dataset.loc[0:2999, :]
test_num = dataset.loc[3000:, :]
test_num = test_num.drop(["revenue_log"], axis=1)

print(train_num.shape)
print(test_num.shape)
#Проверяем есть ли null
print(train_num.isnull().sum().sum())
print(test_num.isnull().sum().sum())

**Обучаем модель**

In [ ]:
X = train_num.drop(['revenue_log'], axis=1).values
y = train_num.revenue_log

In [ ]:
X[1]

In [ ]:
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=False)
print('Train data shape')
print(X_train.shape)
print('Test data shape')
print(X_test.shape)

**Модели**

In [ ]:
# y_test - реальные значения
# y_pred - предсказанные значения
def rmsle(y_test, y_pred):
    return np.sqrt(mean_squared_error(y_test, y_pred))

 
def get_best_score(grid):
    
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)    
    print(grid.best_params_)
    print(grid.best_estimator_)
    
    return best_score

**Linear regression**

In [ ]:
nr_cv = 5 #Кросс-валидация
# verbose - уровень сообщений

linreg = LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False]}
#parameters = {'fit_intercept':[True,False]}
# fit_intercept - Следует ли рассчитывать точку пересечения для этой модели. 
# Если установлено значение False, в расчетах не будет использоваться перехват (т.е. ожидается, что данные будут центрированы).
#  normalize - Этот параметр игнорируется, если fit_interceptустановлено значение False. 
# Если True, регрессоры X будут нормализованы перед регрессией путем вычитания среднего и деления на l2-норму. 
# Если вы хотите стандартизировать, пожалуйста, используйте StandardScalerперед вызовом fit оценщика с normalize=False.
grid_linear = GridSearchCV(linreg, parameters, cv=nr_cv, verbose=1 , scoring = "neg_mean_squared_error")
grid_linear.fit(X, y)

print("best_params", grid_linear.best_params_)
print("Лучший оценщик", grid_linear.best_estimator_)
sc_linear = get_best_score(grid_linear)

In [ ]:
linreg = grid_linear.best_estimator_
linreg.fit(X , y)
pred_linreg = linreg.predict(X_test)


In [ ]:
plt.figure(figsize=(30,10))
plt.plot(np.array(y_test[:100]),label="Реальная")
plt.plot(pred_linreg[:100],label="Предсказанная")
plt.legend(fontsize=15)
plt.title("Значения предсказанной и реальной выручки",fontsize=24)
plt.show()

In [ ]:
rmsle_line = rmsle(y_test,pred_linreg)
print('Cреднеквадратичная логарифмическая ошибка -', rmsle_line)

**KNN Regressor**

In [ ]:
param_grid = {'n_neighbors' : [3,4,5,6,7,8,9,10] }

grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid, cv=nr_cv, refit=True, verbose=1, scoring = "neg_mean_squared_error")
grid_knn.fit(X, y)
print("Лучший оценщик", grid_knn.best_estimator_)
sc_knn = get_best_score(grid_knn)

In [ ]:
KNNreg = grid_knn.best_estimator_
KNNreg.fit(X , y)
pred_KNNreg = KNNreg.predict(X_test)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(np.array(y_test[:100]),label="Реальная")
plt.plot(pred_KNNreg[:100],label="Предсказанная")
plt.legend(fontsize=15)
plt.title("Значения предсказанной и реальной выручки",fontsize=24)
plt.show()

In [ ]:
rmsle_KNN = rmsle(y_test,pred_KNNreg)
print('Cреднеквадратичная логарифмическая ошибка -', rmsle_KNN)

**RandomForestRegressor**

In [ ]:

param_grid = {'min_samples_split' : [3,4,6,10], 'n_estimators' : [50,70,100], 'random_state': [3, 5, 7] }
grid_rf = GridSearchCV(RandomForestRegressor(), param_grid, cv=nr_cv, refit=True, verbose=1, scoring = 'neg_mean_squared_error')
grid_rf.fit(X, y)

print("best_params", grid_rf.best_params_)
print("Лучший оценщик", grid_rf.best_estimator_)

sc_rf = get_best_score(grid_rf)

In [ ]:
rand_for = grid_rf.best_estimator_
rand_for.fit(X , y)


pred_rf = grid_rf.predict(X_test)
#eval_model(grid_rf, 'RandomForestRegressor')

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(np.array(y_test[:100]),label="Реальная")
plt.plot(pred_rf[:100],label="Предсказанная")
plt.legend(fontsize=15)
plt.title("Значения предсказанной и реальной выручки",fontsize=24)
plt.show()

In [ ]:
rmsle_rf = rmsle(y_test,pred_rf)
print('Cреднеквадратичная логарифмическая ошибка -', rmsle_rf)

In [ ]:
#Score  у линейной модели слишком большой - не показываем его на графике
list_scores = [ sc_knn, sc_rf]
list_regressors = [ 'KNN','RF']

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10,7)
sns.barplot(x=list_regressors, y=list_scores, ax=ax)
plt.ylabel('Score')
plt.show()

In [ ]:
list_rmsle = [ rmsle_line, rmsle_KNN, rmsle_rf]
list_regressors = ['Linear','KNN','RF']


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10,7)
sns.barplot(x=list_regressors, y=list_rmsle, ax=ax)
plt.ylabel('RMSLE')
plt.show()

**Запишем результат**

In [ ]:
pred_rf_all = grid_rf.predict(test_num)

sub_rf = pd.DataFrame()

sub_rf['Id'] = test['id']
sub_rf['SalePrice'] = np.exp(pred_rf_all).astype(np.int64)
print(sub_rf.shape)
sub_rf

Графики и score показывают, что алгоритм "Случайный лес" дает меньшую ошибку, но работает дольше